In [2]:
from playwright.async_api import async_playwright
import asyncio

from datetime import datetime, timedelta

### Helper function for get real publish time. 
### Convert Bangla number into English Number

In [5]:
# Mapping Bengali digits to English
bn_digit_map = {
    '০': '0', '১': '1', '২': '2', '৩': '3', '৪': '4',
    '৫': '5', '৬': '6', '৭': '7', '৮': '8', '৯': '9'
}

def bn_to_en_digits(bn_str):
    return ''.join([bn_digit_map.get(ch, ch) for ch in bn_str])
    
###########################################################################
# Function for convert bangla time english and return real published time    
###########################################################################

def parse_bengali_relative_time(bn_time_str):
    if "No time found" in bn_time_str:
        return "No time found"
        
    now = datetime.now()
    
    bn_time_str = bn_time_str.strip()
    
    # Extract number
    num_str = ''.join([bn_digit_map.get(ch, ch) for ch in bn_time_str if ch in bn_digit_map or ch.isdigit()])
    num = int(num_str) if num_str else 0

    if 'সেকেন্ড' in bn_time_str:
        delta = timedelta(seconds=num)
    elif 'মিনিট' in bn_time_str:
        delta = timedelta(minutes=num)
    elif 'ঘণ্টা' in bn_time_str:
        delta = timedelta(hours=num)
    elif 'দিন' in bn_time_str:
        delta = timedelta(days=num)
    else:
        delta = timedelta()  # fallback

    return now - delta

In [7]:
async def fetch_with_playwright():
    news_list = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
        })
        await page.goto("https://www.prothomalo.com/collection/latest", timeout=60000)
        await page.wait_for_selector("#container", timeout=15000)
        #Wait for a card to appear inside container
        await page.wait_for_selector("#container div.news_with_item", timeout=10000)
        elements = await page.locator("h3.headline-title").all()
        for el in elements:
            link_el  =  el.locator("a.title-link")
            href = await link_el.get_attribute("href")
            title = (await el.inner_text()).strip()
            # Get the nearest ancestor or surrounding time tag with class 'published-time'
            time_locator = el.locator("xpath=following::time[contains(@class, 'published-time')]")
        
            # Check if it exists
            if await time_locator.count() > 0:
                published_time = await time_locator.first.inner_text()
            else:
                published_time = "No time found"
            print("published_time", published_time, title)
            news_list.append((title, href, published_time))
            
        await browser.close()
    return news_list
        #for i in range(articles.count())
     

In [10]:
news_list = await fetch_with_playwright() 
#print(news_list)

published_time ৯ মিনিট আগে গ্রীষ্মেও এভাবে হাতের চামড়া উঠছে?
published_time ৯ মিনিট আগে মধ্যরাতে মিরপুর ১ নম্বরে দুই পক্ষের গোলাগুলি, আহত ৩
published_time ১৯ মিনিট আগে ছয় দফা দাবিতে পলিটেকনিক শিক্ষার্থীদের সড়ক অবরোধ
published_time ২০ মিনিট আগে বন্দী করে টাকা আদায়, এরপর তোলা হয় মালয়েশিয়াগামী নৌকায়
published_time ২৩ মিনিট আগে ৪ ঘণ্টা পরও সড়ক ছাড়েননি পলিটেকনিকের শিক্ষার্থীরা, সাতরাস্তাসহ আশপাশে তীব্র যানজট
published_time ২৬ মিনিট আগে ৪ ঘণ্টা পরও সড়ক ছাড়েননি পলিটেকনিকের শিক্ষার্থীরা, সাতরাস্তাসহ আশপাশে তীব্র যানজট
published_time ২৬ মিনিট আগে যে বিরল খনিজ নিয়ে যুক্তরাষ্ট্র-চীন দ্বন্দ্ব, তা পাওয়ার সম্ভাবনা বাংলাদেশে
published_time ২৬ মিনিট আগে ২ জুন বাজেট দেবে অন্তর্বর্তী সরকার
published_time ২৬ মিনিট আগে ২৩ হাজার কোটি টাকায় চার লেন হচ্ছে মহাসড়কটি, তবু থাকছে দুর্ঘটনার ঝুঁকি
published_time ২৬ মিনিট আগে রাজনীতিতে আসা ভুল ছিল না, নির্বাচন করলে আবার জিতব: সাকিব
published_time ২৬ মিনিট আগে মার্কিন উপসহকারী পররাষ্ট্রমন্ত্রীর সঙ্গে বিএনপি, জামায়াত, এনসিপির সৌজন্য সাক্ষাৎ আজ
published_time ২৯ মিনিট 

In [11]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/") # need to on env. for local can't do it right now.
db = client["bd-newspaper-collection"]

collection = db["articles"]

for title,url, published_time in news_list:
    print(title,published_time )
    published_time = parse_bengali_relative_time(published_time)
    print("published_time",published_time)
    if not collection.find_one({"url": url}):
        
        collection.insert_one({
            "title": title,
            "url": url,
            "published_time": published_time,
            "media_name": "prothomalo",
            "scraped_at": datetime.now()
        })
        print(f"Inserted: {title}")
    else:
        print(f"Already Exits {title}")


গ্রীষ্মেও এভাবে হাতের চামড়া উঠছে? ৯ মিনিট আগে
published_time 2025-04-16 14:06:58.404948
Inserted: গ্রীষ্মেও এভাবে হাতের চামড়া উঠছে?
মধ্যরাতে মিরপুর ১ নম্বরে দুই পক্ষের গোলাগুলি, আহত ৩ ৯ মিনিট আগে
published_time 2025-04-16 14:06:58.432950
Inserted: মধ্যরাতে মিরপুর ১ নম্বরে দুই পক্ষের গোলাগুলি, আহত ৩
ছয় দফা দাবিতে পলিটেকনিক শিক্ষার্থীদের সড়ক অবরোধ ১৯ মিনিট আগে
published_time 2025-04-16 13:56:58.437951
Already Exits ছয় দফা দাবিতে পলিটেকনিক শিক্ষার্থীদের সড়ক অবরোধ
বন্দী করে টাকা আদায়, এরপর তোলা হয় মালয়েশিয়াগামী নৌকায় ২০ মিনিট আগে
published_time 2025-04-16 13:55:58.441950
Already Exits বন্দী করে টাকা আদায়, এরপর তোলা হয় মালয়েশিয়াগামী নৌকায়
৪ ঘণ্টা পরও সড়ক ছাড়েননি পলিটেকনিকের শিক্ষার্থীরা, সাতরাস্তাসহ আশপাশে তীব্র যানজট ২৩ মিনিট আগে
published_time 2025-04-16 13:52:58.444950
Already Exits ৪ ঘণ্টা পরও সড়ক ছাড়েননি পলিটেকনিকের শিক্ষার্থীরা, সাতরাস্তাসহ আশপাশে তীব্র যানজট
৪ ঘণ্টা পরও সড়ক ছাড়েননি পলিটেকনিকের শিক্ষার্থীরা, সাতরাস্তাসহ আশপাশে তীব্র যানজট ২৬ মিনিট আগে
published_time 2025-04-16 13:49:5

In [14]:
len(news_list)

25

## This Portion is going to capture highest read article and selected news by prothom alo